In [1]:
import pandas as pd

# coding: utf-8

# 만든 기능 임포트
from crawling import UrlCrawling, ContentCrawling
from preprocessing import Preprocessing
from clustering import Clustering
from summary import Summary
from remove import Remove
from database import insert, selectAll, selectToDay
from tts import tts

c:\Users\USER\Desktop\shortnews\sn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# url 크롤링
url_crawler = UrlCrawling()     # 기사 링크 크롤링 객체

six_url_df = url_crawler.getSixUrl()                            # 6개 카테고리 (카테고리, url 데이터프레임)
entertainment_url_df = url_crawler.getEntertainmentUrl()        # 연예 (카테고리, url 데이터프레임)
sports_url_df = url_crawler.getSportsUrl()                      # 스포츠 (카테고리, url 데이터프레임)

Remove.duplication(six_url_df, entertainment_url_df, sports_url_df)     # 이미 요약한 기사 제거

# 본문 크롤링
six_url, entertainment_url, sports_url = list(six_url_df['six_url']), list(entertainment_url_df['entertainment_url']), list(sports_url_df['sports_url'])
all_url = six_url + entertainment_url + sports_url      # 전체 url
category = list(six_url_df['category']) + list(entertainment_url_df['category']) + list(sports_url_df['category'])

content_crawler = ContentCrawling()     # 기사 크롤링 객체

content_crawler.getSixContent(six_url)                                          # 6개 카테고리 기사 크롤링
content_crawler.getEntertainmentContent(entertainment_url)                      # 연예 기사 크롤링
content_crawler.getSportsContent(sports_url)                                    # 스포츠 기사 크롤링

news_df = content_crawler.makeDataFrame(all_url, category)                      # 본문 데이터프레임 생성

Remove.photoNews(news_df)                                                       # 포토 기사 삭제

Preprocessing.getNouns(news_df)                                                 # 명사 추출

Remove.englishNews(news_df)                                                     # 영어 기사 삭제

vector_list = Preprocessing.getVector(news_df)                                  # 명사 벡터화

Clustering.addClusterNumber(news_df, vector_list)                               # 군집 번호 열 생성
cluster_counts_df = Clustering.getClusteredArticle(news_df)                     # 군집 개수 카운트한 df

summary_news = Summary.getSummaryArticle(news_df, cluster_counts_df)            # 요약한 기사 데이터 프레임 반환

Preprocessing.convertCategory(summary_news)                                     # 카테고리 이름을 번호로 변경

Remove.similarNews(summary_news)                                                # 오늘 요약한 기사와 유사한 기사 drop                       

IT/과학 1 페이지
연예 1 페이지
스포츠 1 페이지
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 
1, 2, 3, 4, 5, 6, 
정치 기사 수 10개 이하
경제 기사 수 10개 이하
사회 기사 수 10개 이하
생활/문화 기사 수 10개 이하
세계 기사 수 10개 이하


In [4]:
del summary_news['naver_summary']
del summary_news['similarity']
del summary_news['test_title']

insert(summary_news.values.tolist())

insert 성공


In [ ]:
# 필요할 때 쓰는 코드
# csv로 저장
summary_news.to_csv("csv/유사도측정.csv",index=False, encoding="utf-8-sig")
df = pd.read_csv("csv/유사도측정.csv")
tts(df)
insert(summary_news.values.tolist())